In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical

In [2]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
import os
from imutils import paths

In [5]:
initial_rate = 1e-4
epochs = 10
batch_size = 32

In [6]:
folder = "dataset"
mask_categories = ["with_mask", "without_mask"]

In [7]:
data = []
labels = []

In [8]:
for category in mask_categories:
    path = os.path.join(folder, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(category)

C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [9]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [10]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [11]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size = 0.30, stratify = labels, random_state = 42)

In [12]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15, horizontal_flip=True, fill_mode="nearest")

In [13]:
baseModel = MobileNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [14]:
hModel = baseModel.output
hModel = AveragePooling2D(pool_size=(7, 7))(hModel)
hModel = Flatten(name="flatten")(hModel)
hModel = Dense(128, activation="relu")(hModel)
hModel = Dropout(0.5)(hModel)
hModel = Dense(2, activation="softmax")(hModel)

In [15]:
model = Model(inputs=baseModel.input, outputs=hModel)

In [16]:
for layer in baseModel.layers:
    layer.trainable = False

In [17]:
model.compile(loss="binary_crossentropy", optimizer = Adam(lr=initial_rate, decay=initial_rate / epochs) , metrics=["accuracy"])

C:\Users\esha kolte\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [18]:
Final_model = model.fit(aug.flow(trainX, trainY, batch_size=batch_size), steps_per_epoch=len(trainX) // batch_size, validation_data=(testX, testY), validation_steps=len(testX) // batch_size, epochs=epochs)

Epoch 1/10
83/83 [==============================] - 103s 1s/step - loss: 0.4399 - accuracy: 0.8265 - val_loss: 0.1732 - val_accuracy: 0.9800
Epoch 2/10
83/83 [==============================] - 90s 1s/step - loss: 0.1715 - accuracy: 0.9578 - val_loss: 0.0884 - val_accuracy: 0.9852
Epoch 3/10
83/83 [==============================] - 90s 1s/step - loss: 0.1034 - accuracy: 0.9717 - val_loss: 0.0653 - val_accuracy: 0.9878
Epoch 4/10
83/83 [==============================] - 90s 1s/step - loss: 0.0861 - accuracy: 0.9740 - val_loss: 0.0522 - val_accuracy: 0.9870
Epoch 5/10
83/83 [==============================] - 90s 1s/step - loss: 0.0744 - accuracy: 0.9808 - val_loss: 0.0530 - val_accuracy: 0.9861
Epoch 6/10
83/83 [==============================] - 89s 1s/step - loss: 0.0722 - accuracy: 0.9762 - val_loss: 0.0438 - val_accuracy: 0.9887
Epoch 7/10
83/83 [==============================] - 88s 1s/step - loss: 0.0522 - accuracy: 0.9864 - val_loss: 0.0398 - val_accuracy: 0.9904
Epoch 8/10
83/83 [=

In [19]:
preIndexs = model.predict(testX, batch_size = batch_size)
preIndexs = np.argmax(preIndexs, axis=1)

In [21]:
print(classification_report(testY.argmax(axis=1), preIndexs,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       575
without_mask       0.99      0.99      0.99       575

    accuracy                           0.99      1150
   macro avg       0.99      0.99      0.99      1150
weighted avg       0.99      0.99      0.99      1150



In [22]:
model.save("mask_detector_trained.model", save_format = "h5")

C:\Users\esha kolte\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
